<a href="https://colab.research.google.com/github/DaeSeokSong/Music-VAE/blob/master/MVAE_assignment_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

## install

In [1]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

#print('Loading pretrained model checkpoints')
#!gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/* .

Installing dependencies...


## Import

### Use pretrained model

In [25]:
import glob

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
    if lib == 'fluidsynth':
        return 'libfluidsynth.so.1'
    else:
        return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

import note_seq

import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Importing libraries and defining some helper functions...
Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


### Use custom model

In [13]:
"""
# Use custom Music VAE 
"""
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.enable_eager_execution()

from google.colab import files

from magenta.scripts import convert_dir_to_note_sequences

from tensorflow.keras import metrics, losses
from tensorflow.keras.optimizers import Adam

from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Funtion

### Use pretrained model

In [27]:
def play(note_sequence):
    mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
    """Interpolates between a start and end sequence."""
    note_sequences = model.interpolate(
        start_seq, end_seq,num_steps=num_steps, length=max_length,
        temperature=temperature,
        assert_same_length=assert_same_length)

    print('Start Seq Reconstruction')
    play(note_sequences[0])

    print('End Seq Reconstruction')
    play(note_sequences[-1])

    print('Mean Sequence')
    play(note_sequences[num_steps // 2])

    print('Start -> End Interpolation')
    interp_seq = mm.sequences_lib.concatenate_sequences(
        note_sequences, [individual_duration] * len(note_sequences))
    
    play(interp_seq)
    mm.plot_sequence(interp_seq)

    return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
    mm.sequence_proto_to_midi_file(note_sequence, filename)
    files.download(filename)

print('Done')

Done


### Use custom model

In [3]:
def _bytes_feature(value):
    """string / byte를 byte_list로 반환합니다."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # ByteList는 EagerTensor의 문자열을 언팩(unpack)하지 않습니다.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """float / double을 float_list로 반환합니다."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """bool / enum / int / unit을 int64_list로 반환합니다."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Extract 4-bar drum sample

## Grobal variable

In [29]:
PRETRAINED_GROOVAE_4BAR = 'groovae_4bar.tar'

## Model

### Load Pretrained Model

In [30]:
# Groove
# 4-bar
groovae_4bar_config = configs.CONFIG_MAP['groovae_4bar']
drums_model = TrainedModel(
    groovae_4bar_config, 
    batch_size=4, 
    checkpoint_dir_or_path=PRETRAINED_GROOVAE_4BAR
    )

### Generate samples as many as sample_num

In [20]:
temperature = 1 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
sample_num = 4 #@param {type:"slider", min:1, max:4, step:1}
drums_samples = drums_model.sample(
    n=sample_num, 
    length=32, 
    temperature=temperature
    )

for idx, ns in enumerate(drums_samples):
    print("Beat ", idx)

    note_seq.plot_sequence(ns)
    play(ns)
    
    print('\n\n')

Beat  0





Beat  1





Beat  2





Beat  3


### download generated MIDI samples.

In [22]:
for i, ns in enumerate(drums_samples):
    download(ns, '%s_sample_%d.mid' % ('groovae_4bar', i))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Build custom Music VAE

## Upload MIDI file

In [4]:
input_midi_data = files.upload().values()

Saving Groove_midi_sample.mid to Groove_midi_sample (1).mid


## Preprocessing

### Writer file that is converted midi to tfrecord

In [8]:
tf_filename = 'groove_midi.tfrecord'
writer = tf.python_io.TFRecordWriter(tf_filename)

convert_dir_to_note_sequences.convert_files('/', 'content/', writer)
writer.close()

INFO:tensorflow:Converting files in '/content/'.
INFO:tensorflow:0 files converted.
INFO:tensorflow:Converted MIDI file /content/Groove_midi_sample.mid.


### Read tfrecord file

In [11]:
row_dataset = tf.data.TFRecordDataset(tf_filename, compression_type="GZIP")
feature_description = {
    'pitch' : tf.io.FixedLenFeature([], tf.int64),
    'velocity' : tf.io.FixedLenFeature([], tf.int64),
    'start_time' : tf.io.FixedLenFeature([], tf.float32),
    'end_time' : tf.io.FixedLenFeature([], tf.float32),
    'instrument' : tf.io.FixedLenFeature([], tf.int64),
    'is_drum' : tf.io.FixedLenFeature([], tf.int64),
}

def _parse_image_function(example_proto):
  # 위의 사전을 이용하여 입력 tf.Example 프로토를 구문 분석합니다.
  return tf.io.parse_single_example(example_proto, feature_description)

parsed_dataset = row_dataset.map(_parse_image_function)

### Restore dataset

In [12]:
dataset = []

for features in parsed_dataset:
    dataset.append(features.numpy())

DataLossError: ignored

## Train

### Model

In [ ]:
model = Sequential()

model.add(Bidirectional(LSTM(64,                        # 해당 층의 노드 개수
                       input_shape=(17, 120, 1),        # input_shape=?
                       return_sequences=True)))         # return_sequences == 각 시퀀스를 출력할지
# model.add(Dropout(0.01))                              # 과적합 방지용 Ex. Dropout 20%(==0.2)
model.add(Dense(32, activation='relu'))
model.add(Bidirectional(LSTM(16)))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='relu'))

model.build(input_shape=(17, 120, 1))
model.compile(loss=losses.MeanSquaredError(),
              optimizer=Adam(learning_rate=0.0001),     # pram ex. learning_rate=0.0001
              metrics=[metrics.MeanSquaredError()]
              )

### Learning

In [ ]:
history = model.fit(X_train, y_train,
                    validation_data = (X_val, y_val),
                    batch_size = 16,                    # Train set's 1~2%, current = 1~2%
                    epochs = 256,                       # Train set's 10%, current = 20%
                    verbose = 1,                        # 0=silent, 1=progress bar, 2=one line per epoch.
                    )

# loss and acc graph (train nd val)
history_df = pd.DataFrame(history.history)
history_df[["loss", "val_loss"]].plot()

# Acc and Loss about real data
learning_lost, learning_err = model.evaluate(X_test, y_test, verbose=2)
print("Learning error % :", learning_err * 100)
print("Learning loss % :", learning_lost * 100)